In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r'D:\fakenews\train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(20800, 5)

In [5]:
#will be only considering title and not text to save time 
#dropping nan values
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df =df.dropna()

In [9]:
df.shape

(18285, 5)

In [11]:
#Get the independent features
x = df.drop('label',axis=1)

In [12]:
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [13]:
y=df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [14]:
x.shape,y.shape

((18285, 4), (18285,))

In [15]:
import tensorflow as tf

In [17]:
tf.__version__

'2.3.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences #to make the sentences of equal lengths
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot #converting sentesnces into onehot given voc size
from tensorflow.keras.layers import LSTM , Dense

In [20]:
#vocabulary size 
voc_size = 10000

In [22]:
#One hot representation
messages = x.copy()

In [25]:
messages.reset_index(inplace = True)

In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rachit.choudhary\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
##Data preprocessing 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [32]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [35]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2906, 3967, 4074, 4927, 4153, 2310, 1257, 2951, 1417, 3748],
 [3971, 2264, 972, 3200, 4982, 4891, 2377],
 [1087, 1342, 620, 77],
 [2749, 1337, 180, 1742, 4630, 1977],
 [2819, 4982, 4890, 4397, 4872, 2585, 4982, 2818, 1119, 2644],
 [593,
  1904,
  2217,
  3379,
  3444,
  1884,
  4586,
  439,
  4355,
  2927,
  4410,
  612,
  4444,
  3407,
  2377],
 [2010, 2226, 2560, 3150, 4303, 3095, 3802, 1202, 3561, 3348, 1164],
 [4539, 2357, 1190, 4720, 4384, 3342, 1884, 217, 3561, 3348, 1164],
 [2609, 2268, 680, 4450, 4843, 1208, 757, 1221, 1884, 114],
 [3689, 4288, 3193, 2976, 1909, 4237, 3646, 1630],
 [1924, 3690, 3575, 2422, 190, 420, 3468, 4905, 3421, 4649, 1169],
 [1742, 2843, 4153, 1208, 1884, 4384],
 [75, 466, 3542, 432, 4322, 4003, 851, 34, 1852],
 [3309, 104, 2106, 1781, 2469, 3262, 3263, 3561, 3348, 1164],
 [3506, 3713, 2834, 350, 2084, 3561, 3348, 1164],
 [2557, 4340, 2236, 847, 1272, 4264, 4803, 4277, 1710, 1855],
 [1082, 1600, 2264],
 [4711, 4923, 841, 943, 1884, 3183, 4264, 2377],
 [

In [38]:
##Embedding Representation 
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)

In [43]:
len(embedded_docs)

18285

In [44]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2906,
       3967, 4074, 4927, 4153, 2310, 1257, 2951, 1417, 3748])

In [46]:
#creating model
embedding_vector_features = 50
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))  #took 1 lstm layer which has 100 neurons
model.add(Dense(1,activation = 'sigmoid')) #added dense layer as it is a classification problem we want 1 answer 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            250000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
len(embedded_docs),y.shape

(18285, (18285,))

In [49]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [50]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [51]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=.33,random_state=42)

In [52]:
#model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 16ms/step - loss: 0.3276 - accuracy: 0.8468 - val_loss: 0.1922 - val_accuracy: 0.9191
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1344 - accuracy: 0.9460 - val_loss: 0.1907 - val_accuracy: 0.9233
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0829 - accuracy: 0.9702 - val_loss: 0.2114 - val_accuracy: 0.9243
Epoch 4/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0492 - accuracy: 0.9828 - val_loss: 0.2465 - val_accuracy: 0.9191
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0267 - accuracy: 0.9907 - val_loss: 0.3096 - val_accuracy: 0.9170
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0163 - accuracy: 0.9951 - val_loss: 0.4271 - val_accuracy: 0.9140
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0102 - accuracy: 0.9971 - val_loss: 0.4539 - val_accuracy: 0.9153

In [54]:
y_pred=model.predict_classes(x_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [57]:
from sklearn.metrics import confusion_matrix

In [59]:
confusion_matrix(y_test,y_pred)

array([[3107,  312],
       [ 198, 2418]], dtype=int64)

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9154929577464789